# Netflix Recommender System

추천 시스템(recommender system)이란 사용자(user)가 상품(item)에 대해 어떻게 평가하는지 예측하는 시스템의 일종이다. Netflix에서 제공한 netflix-prize-data의 경우 17770개의 영화에 대한 고객 2649429명 평점 정보가 저장되어있다. 이 정보를 가지고 사용자가 아직 평가하지 않은 영화에 대한 평점을 예측하는 것이 이번 프로젝트의 목표이다.

# Surprise 패키지

이번 프로젝트에서는 파이썬의 Surprise 패키지를 사용하여 추천 시스템을 구현해 본다. Surprise 패키지는 Nicola Hug가 만든 오픈 소스 추천 시스템 패키로 문서화가 잘 되어 있다.

- https://github.com/NicolasHug/Surprise

# 데이터

넷플릭스에서 제공하는 netflix-prize-data를 사용한다.
- https://www.kaggle.com/netflix-inc/netflix-prize-data


### Netflix-prize-data INFO

CustomerID,Rating,Date
    - MovieIDs range from 1 to 17770 sequentially.
    - CustomerIDs range from 1 to 2649429, with gaps. There are 480189 users.
    - Ratings are on a five star (integral) scale from 1 to 5.
    - Dates have the format YYYY-MM-DD.

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import surprise

## 데이터 전처리

1. netflix-prize-data의 combined_data_1.txt를 오픈
2. surprise 패키지에 사용하기 위한 custom_data_file 생성

In [11]:
data_file = open("/Users/limjungmin/Netflix_Recommender/netflix-prize-data/combined_data_1.txt")
# combined_data_1.txt에는 4499개의 movieID가 저장되어있음.

custom_data_file = open("/Users/limjungmin/Netflix_Recommender/u.data", 'w')

3. 원본데이터의 경우
    
    movieID:
    userID, Rank, Date 의 형태로 저장되어있다.
   
   
4. Surprise 패키지를 사용하기위해 dataset을 custom 해줘야하는데

    userID; movieID; Rank 의 형태로 파일을 수정해야한다.
   
    
   

In [12]:
cnt = 0
for line in data_file:

    if ":" in line:
        movieID = line.split(":")[0]
        #print(movieID)
        cnt+=1
    else :
        
        info = line.split(",")

        userID = info[0]
        rating = info[1]
        date = info[2].split('\n')[0]

    str = userID + ";" + movieID + ";" + rating + "\r\n"
    custom_data_file.write(str)
    
    if cnt > 50 : break
    
print("Done")

Done


## Use a custom dataset
참고: https://surprise.readthedocs.io/en/latest/getting_started.html#use-a-custom-dataset

In [13]:
reader = surprise.Reader(line_format='user item rating', sep=';')

In [14]:
data = surprise.Dataset.load_from_file('/Users/limjungmin/Netflix_Recommender/u.data', reader=reader)

In [15]:
df = pd.DataFrame(data.raw_ratings, columns=["user", "item", "rate", "id"])
del df["id"]

- 포맷 변경후 dataframe으로 변환하여 저장 후 출력 (테스트를 위해 50개의 영화를 대상으로만 진행)

In [17]:
df.head(10)

,user,item,rate
0,1488844,1,3.0
1,822109,1,5.0
2,885013,1,4.0
3,30878,1,4.0
4,823519,1,3.0
5,893988,1,3.0
6,124105,1,4.0
7,1248029,1,3.0
8,1842128,1,4.0
9,2238063,1,3.0


## 평가 

- Surprise 패키지의 cross_validate, KFold를 통해 훈련셋, 테스트셋을 나누어 예측값과 실제값의 비교를 간편히 진행할 수 있다.
- 훈련에 사용한 알고리즘은 netflix prize에서 높은 성능을 보였던 SVD를 사용.

In [18]:
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise import SVD
from surprise import accuracy

In [19]:
kf = KFold(n_splits=3)

In [20]:
algo = SVD()

# 결과
- cross_validate 사용

In [21]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0043  1.0081  1.0044  1.0053  1.0061  1.0056  0.0014  
MAE (testset)     0.7942  0.7977  0.7981  0.7957  0.7919  0.7955  0.0023  
Fit time          15.35   15.19   15.31   15.24   15.34   15.29   0.06    
Test time         0.59    0.44    0.44    0.57    0.55    0.52    0.07    


{'fit_time': (15.349804878234863,
  15.188567638397217,
  15.311560869216919,
  15.239234924316406,
  15.339890241622925),
 'test_mae': array([0.79420578, 0.79772248, 0.79814452, 0.79573241, 0.79193766]),
 'test_rmse': array([1.0042949 , 1.00810347, 1.00435033, 1.00529254, 1.00611877]),
 'test_time': (0.5936009883880615,
  0.437593936920166,
  0.43572211265563965,
  0.5671873092651367,
  0.5539419651031494)}

- KFold 기법 사용

In [22]:
for trainset, testset in kf.split(data):
    
    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 1.0073
RMSE: 1.0043
RMSE: 1.0049


## 보완해야 할 사항

- 전체 데이터 전처리에는 시간소요가 많이 걸리지 않아 기존 병훈이의 데이터 전처리에 비해 월등히 빠름
- 하지만, 현재 movieID 50개를 기준으로만 테스트해본 이유는 전체데이터를 대상으로 할 경우 알고리즘 학습에 많은 시간 소요
- 훈련셋과 테스트셋을 시각화 하여 결측치를 어떤 값으로 예측했지는 좀 더 명확하게 판단하기 위한 모듈 보완
- 더 많은 알고리즘으로 테스트 필요